In [1]:
import pandas as pd
import os 

# Making a path for school and student data
school_data_path = os.path.join('..', 'Resources', 'schools_complete.csv')
students_data_path = os.path.join('..', 'Resources', 'students_complete.csv')

# Reading school and student data into dataframes
school_data = pd.read_csv(school_data_path)
student_data = pd.read_csv(students_data_path)

# Combining both dataframes into one
data_full = pd.merge(student_data, school_data, 'left', ['school_name', 'school_name'])

In [2]:
data_full.columns

Index(['Student ID', 'student_name', 'gender', 'grade', 'school_name',
       'reading_score', 'math_score', 'School ID', 'type', 'size', 'budget'],
      dtype='object')

## District Summary
Calculate the total number of schools, calculate the total number of students, calculate the total budget, calculate the average math score, calculate the average reading score, calculate the percentage of students with a passing math score (70 or greater), calculate the percentage of students with a passing reading score (70 or greater), calculate the percentage of students who passed math **and** reading (% overall passing), and create a dataframe to hold the above results.

In [3]:
# Total number of schools (number of unique schools)
no_schools = data_full['school_name'].nunique()

# Total number of students (number of unique students)
no_students = data_full['Student ID'].nunique()

# Total budget (sum of all schools' budgets)
budget = school_data['budget'].sum()

# Average math score
avg_math_score = data_full['math_score'].mean()

# Average reading score
avg_read_score = data_full['reading_score'].mean()

# Percentage of students with a passing math score (70 or higher)
# Counting the number of students with >= 70 math score
no_pass_math = data_full[data_full['math_score'] >= 70].count()['Student ID']
pass_math = (no_pass_math/no_students) * 100

# Percentage of students with a passing reading score (70 or higher)
# Counting the number of students with >= 70 reading score
no_pass_read = data_full[data_full['reading_score'] >= 70].count()['Student ID']
pass_read = (no_pass_read/no_students) * 100

# Percentage of students who passed math and reading (% overall passing)
# Counting the number of students with >= 70 math and reading scores
no_pass_both = data_full[(data_full['math_score'] >= 70) & (data_full['reading_score'] >= 70)].count()['Student ID']
pass_both = (no_pass_both/no_students) * 100

# Dataframe to hold the above results
district_summary = pd.DataFrame([{'Total Schools': no_schools,
                                 'Total Students': no_students,
                                 'Total Budget': budget,
                                 'Average Math Score': avg_math_score,
                                 'Average Reading Score': avg_read_score,
                                 '% Passing Math': pass_math,
                                 '% Passing Reading': pass_read,
                                 '% Overall Passing': pass_both}])
# Reformatting Total Students and Total Budget 
district_summary['Total Students'] = district_summary['Total Students'].map('{:,}'.format)
district_summary['Total Budget'] = district_summary['Total Budget'].map('${:,.2f}'.format)

In [4]:
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary
For each school, summarize key metrics: school name, school type, total students, total school budget, per student budget, average math score, average reading score, % passing math, % passing reading, and % overall passsing. Create a dataframe to hold the results.

In [24]:
# Making GroupBy based on School Name
by_school = data_full.groupby('school_name')

# School name list
schools = data_full['school_name'].unique()

# School type list
school_types = school_data.set_index('school_name')['type']

# Total students per school list
students = by_school['Student ID'].count()

# Total school budget list
school_budget = by_school['budget'].mean()

# Per student budget list
budget_per_stu = school_budget/students

# Average math score list
avg_math = (by_school['math_score'].sum())/students

# Average reading score list
avg_read = (by_school['reading_score'].sum())/students

# % Passing math list
percent_math = ((data_full[data_full['math_score'] >= 70].groupby('school_name')['math_score'].count())/students) * 100

# % Passing reading list
percent_read = ((data_full[data_full['reading_score'] >= 70].groupby('school_name')['reading_score'].count())/students) * 100

# % Overall passing (passed math and reading) list
percent_overall = ((data_full[(data_full['math_score'] >= 70) & (data_full['reading_score'] >= 70)].groupby('school_name')['reading_score'].count())/students) * 100

# Dataframe to hold the above results
school_summary = pd.DataFrame({'School Type': school_types,
                               'Total Students': students,
                               'Total School Budget': school_budget,
                               'Per Student Budget': budget_per_stu,
                               'Average Math Score': avg_math,
                               'Average Reading Score': avg_read, 
                               '% Passing Math': percent_math,
                               '% Passing Reading': percent_read,
                               '% Overall Passing': percent_overall})
# Reformatting Total School Budget and Per Student Budget + setting School Name as index
# Removing the header 'School Name' above the indicies
school_summary = school_summary.rename_axis(None)
school_summary['Total School Budget'] = school_summary['Total School Budget'].map('${:,.2f}'.format)
school_summary['Per Student Budget'] = school_summary['Per Student Budget'].map('${:,.2f}'.format)

In [25]:
school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Top Performing Schools (By % Overall Passing)
Display the top 5 performing schools by % overall passing.

In [7]:
# Top 5 performing schools by % overall passing (sort by % Overall Passing in descending order)
top = school_summary.sort_values('% Overall Passing', ascending=False)
top.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Pasing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)
Display the bottom 5 performing schools by % overall passing.

In [8]:
# Worst 5 performing schools by % overall passing (sort by % Overall Passing in ascending order)
bottom = school_summary.sort_values('% Overall Passing')
bottom.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Pasing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade
Create a table comparing average math scores by grade for each school.

In [9]:
# For each school and grade level
ninth_m = data_full[data_full['grade'] == '9th'].groupby('school_name')['math_score'].mean()
tenth_m = data_full[data_full['grade'] == '10th'].groupby('school_name')['math_score'].mean()
eleventh_m = data_full[data_full['grade'] == '11th'].groupby('school_name')['math_score'].mean()
twelfth_m = data_full[data_full['grade'] == '12th'].groupby('school_name')['math_score'].mean()

# Combine series into dataframe
math_by_grade = pd.DataFrame({'9th': ninth_m,
                             '10th': tenth_m,
                             '11th': eleventh_m,
                             '12th': twelfth_m})
# Reformatting the dataframe (removing the axis header)
math_by_grade = math_by_grade.rename_axis(None)

In [10]:
math_by_grade

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade
Create a table comparing average reading scores by grade for each school.

In [11]:
# For each school and grade level
ninth_r = data_full[data_full['grade'] == '9th'].groupby('school_name')['reading_score'].mean()
tenth_r = data_full[data_full['grade'] == '10th'].groupby('school_name')['reading_score'].mean()
eleventh_r = data_full[data_full['grade'] == '11th'].groupby('school_name')['reading_score'].mean()
twelfth_r = data_full[data_full['grade'] == '12th'].groupby('school_name')['reading_score'].mean()

# Combine series into dataframe
reading_by_grade = pd.DataFrame({'9th': ninth_r,
                             '10th': tenth_r,
                             '11th': eleventh_r,
                             '12th': twelfth_r})
# Reformatting the dataframe (removing the axis header)
reading_by_grade = reading_by_grade.rename_axis(None)

In [12]:
reading_by_grade

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending
Using 4 reasonable bins to group school spending, make a table including average math score, average reading score, % passing math, % passing reading, and % overall passing.

In [26]:
# Bins at 0, 585, 630, 645, and 680 
bins_spending = [0, 585, 630, 645, 680]
# Group names for the bins (spending ranges per student)
spending_groups = ['<$585', '$585-630', '$630-645', '$645-680']

# Adding Spending Ranges (Per Student) onto school_summary dataframe using bins_spending and spending_names based on Per Student Budget
school_summary['Spending Ranges (Per Student)'] = pd.cut(budget_per_stu, bins_spending, labels=spending_groups, right=True)

# Average math score per category
avg_math_spending = round(school_summary.groupby('Spending Ranges (Per Student)')['Average Math Score'].mean(), 2)

# Average reading score per category
avg_read_spending = round(school_summary.groupby('Spending Ranges (Per Student)')['Average Reading Score'].mean(), 2)

# % Passing math per category
percent_math_spending = round(school_summary.groupby('Spending Ranges (Per Student)')['% Passing Math'].mean(), 2)

# % Passing reading per category
percent_read_spending = round(school_summary.groupby('Spending Ranges (Per Student)')['% Passing Reading'].mean(), 2)

# % overall passing per category
overall_percent_spending = round(school_summary.groupby('Spending Ranges (Per Student)')['% Overall Passing'].mean(), 2)

# Creating dataframe based on above data
scores_spending = pd.DataFrame({'Average Math Score': avg_math_spending,
                               'Average Reading Score': avg_read_spending,
                               '% Passing Math': percent_math_spending,
                               '% Passing Reading': percent_read_spending,
                               '% Overall Passing': overall_percent_spending})

In [27]:
scores_spending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


## Scores by School Size
Using 3 reasonable bins for school size, create a table including average math score, average reading score, % passing math, % passing reading, and % overall passing.

In [28]:
# Bins at 0, 1000, 2000, and 5000
bins_size = [0, 1000, 2000, 5000]
# Group names for the bins (school size)
school_size = ['Small (<1000)', 'Medium (1000-2000)', 'Large (2000-5000)']

# Adding Spending Ranges (Per Student) onto school_summary dataframe using bins_spending and spending_names based on Per Student Budget
school_summary['School Size'] = pd.cut(students, bins_size, labels=school_size, right=True)

# Average math score per category
avg_math_size = round(school_summary.groupby('School Size')['Average Math Score'].mean(), 2)

# Average reading score per category
avg_read_size = round(school_summary.groupby('School Size')['Average Reading Score'].mean(), 2)

# % Passing math per category
percent_math_size = round(school_summary.groupby('School Size')['% Passing Math'].mean(), 2)

# % Passing reading per category
percent_read_size = round(school_summary.groupby('School Size')['% Passing Reading'].mean(), 2)

# % overall passing per category
overall_percent_size = round(school_summary.groupby('School Size')['% Overall Passing'].mean(), 2)

# Creating dataframe based on above data
scores_size = pd.DataFrame({'Average Math Score': avg_math_size,
                               'Average Reading Score': avg_read_size,
                               '% Passing Math': percent_math_size,
                               '% Passing Reading': percent_read_size,
                               '% Overall Passing': overall_percent_size})

In [29]:
scores_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.82,83.93,93.55,96.10,89.88
Medium (1000-2000),83.37,83.86,93.60,96.79,90.62
Large (2000-5000),77.75,81.34,69.96,82.77,58.29


## Scores by School Type
Grouped by school type, create a table containing average math score, average reading score, % passing math, % passing reading, and % overall passing.

In [30]:
# Average math score per category
avg_math_type = round(school_summary.groupby('School Type')['Average Math Score'].mean(), 2)

# Average reading score per category
avg_read_type = round(school_summary.groupby('School Type')['Average Reading Score'].mean(), 2)

# % Passing math per category
percent_math_type = round(school_summary.groupby('School Type')['% Passing Math'].mean(), 2)

# % Passing reading per category
percent_read_type = round(school_summary.groupby('School Type')['% Passing Reading'].mean(), 2)

# % overall passing per category
overall_percent_type = round(school_summary.groupby('School Type')['% Overall Passing'].mean(), 2)

# Creating dataframe based on above data
scores_type = pd.DataFrame({'Average Math Score': avg_math_type,
                               'Average Reading Score': avg_read_type,
                               '% Passing Math': percent_math_type,
                               '% Passing Reading': percent_read_type,
                               '% Overall Passing': overall_percent_type})

In [31]:
scores_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,93.62,96.59,90.43
District,76.96,80.97,66.55,80.80,53.67


## Conclusions
The most startling difference in test scores (math and overall) occurs between Charter and District type schools. While their average math and reading scores both differ by less than 10 points, there is a large discrepancy between the percentage of students who pass math and pass overall between the two school types. Overall, just above half (53.67%) of the students at a District school pass both math and reading, contrasting the 90.43% who pass at a Charter school. 

There also appears to be a correlation between school size and a decreasing overall passing rate, however the correlation is not linear. Both small schools (less than 1000 students) and medium schools (1000-2000 students) have around a 90% overall passing rate - 89.88% and 90.62%, respectively. As the school size increases to over 2000, the overall passing rate drops to 58.29%. Fewer resources available to a large school density can be attributed to this drop. This trend is depicted in the Bottom Performing Schools table, which shows that the bottom 5 performing schools all have greater than 2000 students. 